In [1]:
import numpy as np
from heapq import heappush, nlargest
import warnings
import unicodedata, re
try:
    from sklearn.metrics.pairwise import cosine_similarity
    usesklearn = True
except ImportError:
    usesklearn = False
try:
    import networkx as nx
except ImportError:
    warnings.warn("Could not import networkx. Constructing cluster network is not available")
    pass


from itertools import product

from TurkishStemmer import TurkishStemmer
from textblob import TextBlob
from textblob.exceptions import NotTranslated

import gensim, pickle

class Cluster:
    def __init__(self, samples=5, keep_authors_status=False):
        self.cluster_vector = None
        self.vectors_sim = None  # [(similarity, np.zeros(vector_size))]
        self.vectors = list()  # [list of cluster vectors]
        self.authors = dict()
        self.keep_authors_status = keep_authors_status
        self.samples = [None for i in range(samples)]
        self._nonesamples = samples-1

    def root_similarity(self, v1):
        """
            similarity_to_cluster_vector
        :param v1: np 1-D array
        :return: similarity to the cluster vector
        """
        return self.cos_sim(v1, self.cluster_vector)

    def get_top_n(self, n=5):
        """
        :param n: number of most similar vectors
        :return: most similar n vectors to that cluster
        """
        ## TODO return items
        if n > len(self.vectors):
            warnings.warn("n is bigger than the number of vectors in that cluster")
        return nlargest(min(n, len(self.vectors)), self.vectors_sim)

    @staticmethod
    def cos_sim(v1, v2):
        if usesklearn:
            return np.float64(cosine_similarity(np.atleast_2d(v1),np.atleast_2d(v2)))
        else:
            return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

    def update_cluster_vector(self):
        self.vectors_sim = list()
        self.cluster_vector = np.mean(np.array(self.vectors), axis=0)
        for vector in self.vectors:
            heappush(self.vectors_sim, (self.root_similarity(vector), vector))
            # self.vectors_sim.append((self.root_similarity(vector), vector))
        assert self.cluster_vector is not None
    
    def addtext(self, text, author):
        if self.keep_authors_status and author:
            self.authors.setdefault(author, list())
            self.authors[author].append(text)
        else:
            self.authors.setdefault(author, 0)
            self.authors[author] += 1
        
        if self._nonesamples >= 0:
            self.samples[self._nonesamples] = text
            self._nonesamples -= 1
        elif np.random.choice([True, False]):
            self.samples[np.random.randint(0, len(self.samples))] = text
    
    def add(self, vector, text, author):
#         try:
        self.vectors.append(vector)
        self.addtext(text, author)
        self.update_cluster_vector()
#         except:
#             warnings.warn("An error occured during updating the cluster metrics. Last changes reveresed.")
#             return True

    def __hash__(self):
        h = hash(str(self.cluster_vector))
        for i in self.vectors[:min(len(self.vectors), 3)]:
            h *= hash(str(i))
        return h


class AdaptiveOnlineClustering:

    def __init__(self, en_w2v, tr_w2v, similarity_threshold=0.75, cluster_nsamples=5, vector_size=300):
        self.en_w2v = en_w2v
        self.tr_w2v = tr_w2v
        self.vector_size = vector_size
        self.turkish_stemmer = TurkishStemmer()
        self.similarity_threshold = similarity_threshold
        self.cluster_nsamples = cluster_nsamples
        self.clusters = dict()

    def add(self, text, language="en", author=None, translate=True, stem=False):
        vec = self.vectorize(text, language, translate=translate, stem=stem)
        if vec is None:
            warnings.warn("Invalid text. Document skipped")
        else:
            self._add(vec, text, author)

    def _add(self, vector, text, author):
        highest_similarity = 0
        assigned_cluster = None
        for cluster in self.clusters:
            sim = self.clusters[cluster].root_similarity(vector)
            if sim > highest_similarity:
                highest_similarity = sim
                assigned_cluster = cluster
        if highest_similarity >= self.similarity_threshold:
            self.clusters[assigned_cluster].add(vector, text, author)
        else:
            new_cluster = Cluster(self.cluster_nsamples)
            added = new_cluster.add(vector, text, author)
            self.clusters[len(self.clusters)] = new_cluster
        self._update_clusters()

    def _update_clusters(self):
        for cluster in self.clusters:
            if len(self.clusters[cluster].vectors) < 1:
                del self.clusters[cluster]

    def vectorize(self, text, language, translate=True, stem=False):
        blob = self.clean(text, language, translate=translate, stem=stem)
        if not blob:
            return
        vector = np.zeros(self.vector_size)
        if len(blob.words) < 1:
            return None

        for word in blob.words:
            try:
                if language == "en" or translate:
                    vector += self.en_w2v[word]
                else:
                    vector += self.tr_w2v[word]
            except KeyError:
                continue
        vector /= len(blob.words)
        return vector

    def clean(self, text, language="en", translate=True, stem=False):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').lower().decode("ascii")
        # if language == "tr":
        #     if stem:
        #         text= ' '.join([self.turkish_stemmer.stem(w) for w in text.split()])
        blob = TextBlob(text)
        if translate and language != "en":
            try:
                blob = blob.translate(to="en")
            except NotTranslated:
                return 
        text = str(blob)
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r'[0-9]', '#', text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ", text)
        text = re.sub(r"\+", " ", text)
        text = re.sub(r"\-", " ", text)
        text = re.sub(r"\=", " ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r":", " ", text)
        text = re.sub(r"e(\s)?-(\s)?mail", "email", text)

        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        return TextBlob(text)

    def get_network_graph(self, print_prob=0.0):
        network_dict = dict() # {(from,to): weight}
        for c in self.clusters.values():
            for f, t in product(c.authors, c.authors):
                if f==t: continue
                if (f,t) in network_dict:
                    network_dict[(f,t)] += 1
                elif (t,f) in network_dict:
                    network_dict[(t,f)] += 1
                else:
                    network_dict[(f,t)] = 1
        g = nx.Graph()
        for (f,t), weight in network_dict.items():
            if np.random.choice(a=[False, True], p=[1-print_prob, print_prob]):
                print(f,t,weight)
            g.add_edge(f, t, weight=weight)
        return g

In [2]:
tweets = pickle.load(open('../datasets/tweets.pkl', 'rb'))

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('/home/ammar/NLP_data/GoogleNews-vectors-negative300.bin', binary=True)  

In [4]:
a = AdaptiveOnlineClustering(model, None)

In [5]:
import time
start = time.time()
for author in tweets[:10]:
    for tweet in author[:10]:
#         if i.lang == "en":
#         print(i.author.id)
        a.add(tweet.text, tweet.lang, tweet.author.id)
print("Took {} seconds".format(time.time()-start))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:112: UserWarning: Invalid text. Document skipped


Took 15.009860038757324 seconds


In [6]:
print("There are {} clusters of 24 documents".format(len(a.clusters)))
for e,c in a.clusters.items():
#     print(c.samples)
    print("Cluster {}:".format(e))
    print("Number of samples:",sum([i is not None for i in c.samples]))
    print("Number of tweets:",len(c.vectors))
    print("Number of authors:",len(c.authors))
    print("Authors IDs > # of their tweets in that cluster: \n", c.authors)
    print()
#     print(c.samples[0] is None)

There are 37 clusters of 24 documents
Cluster 0:
Number of samples: 5
Number of tweets: 25
Number of authors: 9
Authors IDs > # of their tweets in that cluster: 
 {765555429640470528: 2, 795688128: 1, 927954050: 6, 2290443619: 2, 2667641060: 2, 341498661: 1, 2386861894: 3, 3879123083: 4, 3068366386: 4}

Cluster 1:
Number of samples: 3
Number of tweets: 3
Number of authors: 2
Authors IDs > # of their tweets in that cluster: 
 {919580399847858176: 2, 927954050: 1}

Cluster 2:
Number of samples: 1
Number of tweets: 1
Number of authors: 1
Authors IDs > # of their tweets in that cluster: 
 {927954050: 1}

Cluster 3:
Number of samples: 1
Number of tweets: 1
Number of authors: 1
Authors IDs > # of their tweets in that cluster: 
 {341498661: 1}

Cluster 4:
Number of samples: 1
Number of tweets: 1
Number of authors: 1
Authors IDs > # of their tweets in that cluster: 
 {341498661: 1}

Cluster 5:
Number of samples: 1
Number of tweets: 1
Number of authors: 1
Authors IDs > # of their tweets in that

In [7]:
graph = a.get_network_graph(print_prob=0.5)

765555429640470528 2290443619 4
2290443619 3879123083 4
765555429640470528 795688128 4
2667641060 341498661 2
795688128 3068366386 4
795688128 341498661 2
795688128 2667641060 4
795688128 3879123083 4
927954050 341498661 2
2290443619 341498661 2
2386861894 3068366386 4
341498661 3068366386 2
795688128 927954050 2
341498661 2386861894 2
927954050 3068366386 2
927954050 2667641060 2
795688128 2386861894 4
765555429640470528 3879123083 4
765555429640470528 341498661 2
927954050 2290443619 2
2290443619 2667641060 4
765555429640470528 927954050 2
